In [5]:
import pandas as pd

file_path = "LiveLab_top_15_MATHia_7x_classes_anon.csv"

df = pd.read_csv(file_path)

print(df.head(2))

# display column names
print("\nColumn names:", df.columns.tolist())


  Anon Student Id                                 Session Id  \
0            s141  qb8uqykv9llqylywx-ilzwqrqn0:1679667026583   
1            s141  qb8uqykv9llqylywx-ilzwqrqn0:1679667026583   

                  Time                         KC_Model(MATHia)  \
0  2023-03-24 14:26:50  match _dep expression with description.   
1  2023-03-24 14:26:53  match _dep expression with description.   

          Level (Section)                Problem Name           Step Name  \
0  analyzing_models_1step  analyzing_models_1step-029  gn-classify-item-4   
1  analyzing_models_1step  analyzing_models_1step-029  gn-classify-item-2   

   Attempt At Step  Help Level  Selection  ... CF (Skill Previous p-Known)  \
0                1           0        NaN  ...                    0.650237   
1                1           0        NaN  ...                    0.887118   

  CF (Skill New p-Known)                           CF (Module)  \
0               0.887118  determining_unknown_quantities_mb_c1   
1     

In [9]:
# Count how many transactions in each class
cf_class_id_counts = df['cf_class_id'].value_counts()

print(cf_class_id_counts)

cf_class_id
c8     213291
c12    183133
c2     159826
c15    143542
c9     138025
c7     130800
c6     130553
c13    130237
c10     98389
c5      85363
c3      76204
c14     62286
c11     55127
c4      25719
c1      17851
Name: count, dtype: int64


In [19]:
# Group by 'cf_class_id', then 'Anon Student Id', then 'Session Id' and count occurrences
cf_class_id_student_session_counts = df.groupby(['cf_class_id', 'Anon Student Id', 'Session Id']).size()

print(cf_class_id_student_session_counts)

cf_class_id_student_session_counts.to_csv("cf_class_id_student_session_counts.csv", header=True)

cf_class_id  Anon Student Id  Session Id                               
c1           s111             02nz11rk5vapthxnhkufh2mlx7q:1677166192418    225
                              8jnruvmdjomeujh343gnd9aw_f0:1666797238899     74
                              eirroqbeufsh8kymyguof0qkrfg:1666712116267     75
                              hpqwpqrwr3o0egjnrmmans2nibq:1672932855145     78
                              kiteau2dhnkmlvmwowpsjixeiok:1667315656734     41
                                                                          ... 
c9           s61              sggvqtnhbds1gc9e7ag_tt_cvy4:1673283183502      7
                              tucwr3pcny0hzaiv_0mkalvgrkq:1679587647015      5
                              wbxdxftpwe70qoyyvnwxwzrh6fw:1678118495644      4
                              xvski0imfubiuzrvmh_tf2pz6jo:1674060267788      5
                              ykpo9hgz_c7tbycmxso4ddo18ci:1679326599181    231
Length: 15517, dtype: int64


In [ ]:
#Single session ——> for IDEL

In [41]:
import pandas as pd

file_path = "single_session.csv"
df = pd.read_csv(file_path)

# Convert 'Time' column to datetime format with automatic inference
df['Time'] = pd.to_datetime(df['Time'], format='mixed', errors='coerce')

# Sort by time to ensure chronological order
df = df.sort_values(by='Time')

# Calculate time difference with the previous row
df['Time_Diff_Minutes'] = df['Time'].diff().dt.total_seconds() / 60  # Convert seconds to minutes

# Define 'Idle' column (True if the time difference > 5 minutes)
df['Idle'] = df['Time_Diff_Minutes'] > 5

output_file = "single_session_processed.csv"
df.to_csv(output_file, index=False)

In [ ]:
#Single session ——> for MONITOR

In [39]:
# Function to calculate rolling window statistics
def rolling_monitor_status(outcomes):
    monitor_status = []  # Stores True/False for each row
    attempt_counts = []  # Stores Attempt count for each row
    for i in range(len(outcomes)):
        # Get the last 10 rows up to the current row
        last_10 = outcomes[max(0, i - 9): i + 1].astype(str)
        
        # Count occurrences
        attempt_count = last_10.str.count("Attempt").sum()
        ok_count = last_10.str.count("OK").sum()

        # First 10 rows should have Monitor = False
        if i < 9:
            monitor_status.append(False)
        else:
            monitor_status.append(ok_count <= 3)  # True if "OK" appears ≤ 3 times
        
        attempt_counts.append(attempt_count)
    
    return pd.DataFrame({'Monitor': monitor_status, 'Attempt_Count': attempt_counts})

# Apply function for each student independently
df[['Monitor', 'Attempt_Count']] = df.groupby('Anon Student Id')['Outcome'].apply(rolling_monitor_status).reset_index(drop=True)


output_file = "single_session_with_monitor_attempts.csv"
df.to_csv(output_file, index=False)
